In [1]:
import os
import shutil

from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.core import Environment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.33.0 (c:\users\anildwa\miniconda3\envs\aml\lib\site-packages), Requirement.parse('azureml-core~=1.32.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-core 1.33.0 (c:\users\anildwa\miniconda3\envs\aml\lib\site-packages), Requirement.parse('azureml-core~=1.32.0'), {'azureml-telemetry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.33.0 (c:\users\anildwa\miniconda3\envs\aml\lib\site-packages), Requirement.parse('azureml-core~=1.32.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_r

In [2]:
ws = Workspace.from_config()
project_folder = './scripts'

In [3]:
cluster_name = "k80cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target


In [4]:
curated_env_name = 'AzureML-PyTorch-1.6-GPU'
pytorch_env = Environment.get(workspace=ws, name=curated_env_name)

In [5]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder,
                      script='pytorch_train.py',
                      arguments=['--num_epochs', 30, '--output_dir', './outputs'],
                      compute_target=compute_target,
                      environment=pytorch_env)

In [6]:
import os
from azureml.pipeline.steps import CommandStep
train_config = ScriptRunConfig(source_directory=project_folder,
                      command=['python', 'pytorch_train.py','--num_epochs', 30, '--output_dir', './outputs'],
                      compute_target=compute_target,
                      environment=pytorch_env)
    
train_step = CommandStep(name='train-step', inputs=[],
                      outputs=[], runconfig=train_config)

In [7]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(workspace=ws, steps=[train_step])

In [16]:
pipeline_run = Experiment(ws, 'pytorch-pipeline').submit(pipeline)

Created step train-step [9ce7dbce][64886203-5ec8-4987-a314-e668c9d8627e], (This step will run and generate new outputs)
Submitted PipelineRun 2a51c169-5ee2-42d3-a585-37e82513eb78
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2a51c169-5ee2-42d3-a585-37e82513eb78?wsid=/subscriptions/3e0e14b3-7e28-4da7-97de-0f5cb324f030/resourcegroups/ml/workspaces/ml-service&tid=72f988bf-86f1-41af-91ab-2d7cd011db47


In [17]:
run.wait_for_completion(show_output=True)

RunId: pytorch-pipeline_1616994052_0fa808a2
Web View: https://ml.azure.com/runs/pytorch-pipeline_1616994052_0fa808a2?wsid=/subscriptions/3e0e14b3-7e28-4da7-97de-0f5cb324f030/resourcegroups/ml/workspaces/ml-service&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Execution Summary
RunId: pytorch-pipeline_1616994052_0fa808a2
Web View: https://ml.azure.com/runs/pytorch-pipeline_1616994052_0fa808a2?wsid=/subscriptions/3e0e14b3-7e28-4da7-97de-0f5cb324f030/resourcegroups/ml/workspaces/ml-service&tid=72f988bf-86f1-41af-91ab-2d7cd011db47



{'runId': 'pytorch-pipeline_1616994052_0fa808a2',
 'target': 'k80cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-29T05:04:36.654153Z',
 'endTimeUtc': '2021-03-29T05:09:20.31035Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '11122060-740b-4cbd-b201-defc0b8b7d3d',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'pytorch_train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--num_epochs', '30', '--output_dir', './outputs'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'k80cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'AzureML-PyTorch-1.

In [9]:
run = Experiment(ws, name='pytorch-pipeline').submit(src)
run.wait_for_completion(show_output=True)

RunId: pytorch-pipeline_1616994052_0fa808a2
Web View: https://ml.azure.com/runs/pytorch-pipeline_1616994052_0fa808a2?wsid=/subscriptions/3e0e14b3-7e28-4da7-97de-0f5cb324f030/resourcegroups/ml/workspaces/ml-service&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/55_azureml-execution-tvmps_c3e9b6501534b7e2fc80cbe51460ffce05dfe9b0553a05383f0d64b2b51ab230_d.txt

2021-03-29T05:04:37Z Starting output-watcher...
2021-03-29T05:04:37Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-03-29T05:04:37Z Executing 'Copy ACR Details file' on 10.0.0.4
2021-03-29T05:04:37Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_31eff05d9938cbbdf21dfb15bccdb84f
171857c49d0f: Pulling fs layer
419640447d26: Pulling fs layer
61e52f862619: Pulling fs layer
c118dad7e37a: Pulling fs layer
2e36372995f9: Pulling fs layer
0b8e00a4ba4e: Pulling fs layer
b3026b4f2581: Pulling fs layer
93cc19

{'runId': 'pytorch-pipeline_1616994052_0fa808a2',
 'target': 'k80cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-29T05:04:36.654153Z',
 'endTimeUtc': '2021-03-29T05:09:20.31035Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '11122060-740b-4cbd-b201-defc0b8b7d3d',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'pytorch_train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--num_epochs', '30', '--output_dir', './outputs'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'k80cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'AzureML-PyTorch-1.